In [1]:
import os, sys
from python_tools import *

python tools loaded.


In [2]:
#take in hadron list, and calculate basic properties
#return theta_xz, theta_yz, theta_z, ke
def calc_hadrons(pdg=None,px=None,py=None,pz=None,energy=None,s=None):
    
    if s is not None:
        pdg = s[0]
        px = s[1]
        py = s[2]
        pz = s[3]
        energy = s[4]

    thetaxz = np.arctan2(px,pz)
    thetayz = np.arctan2(py,pz)
    thetaz = np.arctan2(np.sqrt(np.square(px)+np.square(py)),pz)
    ke = np.array( [ (energy[i] - particle.Particle.from_pdgid(pdg[i]).mass*1e-3) for i in range(len(pdg)) ],
                   dtype=np.dtype(np.float) )
    
    return thetaxz,thetayz,thetaz,ke

In [3]:
#utility function to get the max element of a sub-array
def get_max_val_by_mask(val_arr,mask):
    sub_idx = np.argmax(val_arr[mask])
    max_idx = np.arange(val_arr.shape[0])[mask][sub_idx]
    return max_idx

In [4]:
#apply an acceptance based on particle type, angle, and KE thresholds
#return acceptance for hadron list, number of passing hadrons, and properties of leading (by KE) hadron
def acceptance(pdg=None,px=None,py=None,pz=None,energy=None,thetaz=None,ke=None,
               pdgcode=2212,thetaz_thresh_deg=40,ke_thresh_mev=100.,
               s=None):
    
    if s is not None:
        pdg = s[0]
        px = s[1]
        py = s[2]
        pz = s[3]
        energy = s[4]
        thetaz = s[5]
        ke = s[6]
    
    #put in rad and GeV
    THETAZ_THRESH_RAD=np.radians(thetaz_thresh_deg)
    KE_THRESH_GEV=ke_thresh_mev*1e-3

    mask = (pdg==pdgcode)
    accept = np.logical_and(mask,np.abs(thetaz)<THETAZ_THRESH_RAD)
    
    px_1=np.nan
    py_1=np.nan
    pz_1=np.nan
    E_1=np.nan
    ke_1=np.nan
    thetaz_1=np.nan
    if np.any(accept):
        max_idx = get_max_val_by_mask(ke,accept)
        px_1=px[max_idx]
        py_1=py[max_idx]
        pz_1=pz[max_idx]
        E_1=energy[max_idx]
        ke_1=ke[max_idx]
        thetaz_1=thetaz[max_idx]
            
    return accept,np.sum(np.logical_and(accept,ke>KE_THRESH_GEV)),px_1,py_1,pz_1,E_1,ke_1,thetaz_1

In [5]:
#function for mergning in event weights
def merge_weights(file,tree_names,var_suffix,df):
    for i in range(len(tree_names)):
        w_df = uproot.open(file)[tree_names[i]].pandas.df()
        wght_name="wght_"+var_suffix[i]
        w_df = w_df.rename(columns={"wght": wght_name})
        df = df.merge(w_df[["iev",wght_name]],how="left",on=["iev"])
    return df

In [6]:
#function for combining all the event weights from the root files I've made...
def merge_weights_simple(filepath,knob_name,df,qual=""):
    filename="%s/weights_%s_%s.root"%(filepath,knob_name,qual)
    tree_names = [ "%s_twk%d"%(knob_name,i) for i in [0,1,3,4] ]
    var_suffixes = [ "%s_n2"%knob_name,"%s_n1"%knob_name,"%s_p1"%knob_name,"%s_p2"%knob_name ]
    return merge_weights(filename,tree_names,var_suffixes,df)

In [7]:
#functions for some lepton quantities
def add_lepton_calcs(df):
    df['ptl'] = np.sqrt(np.square(df['pxl'])+np.square(df['pyl']))
    df['thetal'] = np.arctan2(df['ptl'],df['pzl'])

In [8]:
#functions for performing hadron calcs and acceptances
def add_hadron_calcs(df):
    df['thetaxzf'], df['thetayzf'], df['thetazf'], df['kef'] \
        = zip(*df[["pdgf","pxf","pyf","pzf","Ef"]].apply((lambda x: calc_hadrons(s=x)),axis=1))
    
def add_neutron_acceptance(df):
    df['accept_n'], df['n_n'], df['px_n1'], df['py_n1'], df['pz_n1'], df['E_n1'], df['ke_n1'], df['thetaz_n1'] \
        = zip(*df[["pdgf","pxf","pyf","pzf","Ef","thetazf","kef"]].apply((lambda x: acceptance(s=x,
                                                                                               pdgcode=2112,
                                                                                               thetaz_thresh_deg=360.,
                                                                                               ke_thresh_mev=0.)),
                                                                         axis=1))

def add_proton_acceptance(df):
    df['accept_p'], df['n_p'], df['px_p1'], df['py_p1'], df['pz_p1'], df['E_p1'], df['ke_p1'], df['thetaz_p1'] \
        = zip(*df[["pdgf","pxf","pyf","pzf","Ef","thetazf","kef"]].apply((lambda x: acceptance(s=x,
                                                                                               pdgcode=2212,
                                                                                               thetaz_thresh_deg=360.,
                                                                                               ke_thresh_mev=0.)),
                                                                         axis=1))


In [9]:
def create_base_df(root_filename,out_name=None,max_entries=-1,verbose=True):
    
    print("Reading %d entries from root file %s"%(max_entries,root_filename))
    gst_df = uproot.open(root_filename)['gst'].pandas.df(flatten=False)
    
    if(max_entries>0):
        gst_df = gst_df.loc[:max_entries]
    
    print("Adding lepton quantities.")
    #lepton quantities
    add_lepton_calcs(gst_df)
    
    print("Adding hadron quantities.")
    #hadron quantities
    add_hadron_calcs(gst_df)
    
    print("Adding hadron acceptances.")
    #acceptances
    add_neutron_acceptance(gst_df)
    add_proton_acceptance(gst_df)
    
    if(out_name):
        print("Writing output file %s"%out_name)
        gst_df.to_hdf(out_name,key="gst_df")
    
    return gst_df

In [11]:
gst_df = uproot.open("/Users/wketchum/Data/LDMX/gntp.0.gst.root")['gst'].pandas.df(flatten=False)

In [13]:
gst_df.columns

Index(['iev', 'neu', 'fspl', 'tgt', 'Z', 'A', 'hitnuc', 'hitqrk', 'resid',
       'sea', 'qel', 'mec', 'res', 'dis', 'coh', 'dfr', 'imd', 'imdanh',
       'singlek', 'nuel', 'em', 'cc', 'nc', 'charm', 'amnugamma', 'neut_code',
       'nuance_code', 'wght', 'xs', 'ys', 'ts', 'Q2s', 'Ws', 'x', 'y', 't',
       'Q2', 'W', 'EvRF', 'Ev', 'pxv', 'pyv', 'pzv', 'En', 'pxn', 'pyn', 'pzn',
       'El', 'pxl', 'pyl', 'pzl', 'pl', 'cthl', 'nfp', 'nfn', 'nfpip', 'nfpim',
       'nfpi0', 'nfkp', 'nfkm', 'nfk0', 'nfem', 'nfother', 'nip', 'nin',
       'nipip', 'nipim', 'nipi0', 'nikp', 'nikm', 'nik0', 'niem', 'niother',
       'ni', 'pdgi', 'resc', 'Ei', 'pxi', 'pyi', 'pzi', 'nf', 'pdgf', 'Ef',
       'pxf', 'pyf', 'pzf', 'pf', 'cthf', 'vtxx', 'vtxy', 'vtxz', 'vtxt',
       'sumKEf', 'calresp0'],
      dtype='object')

In [21]:
gst_df[0:10][["iev","ni","pdgi","resc","Ei","pxi","pyi","pzi"]]

,iev,ni,pdgi,resc,Ei,pxi,pyi,pzi
entry,,,,,,,,
0,0,2,"[2112, 111]","[1, 7]","[0.9617546426360942, 0.6555073152025024]","[0.0457321474779242, -0.14466688283247225]","[-0.03015256614920707, -0.1162190789148202]","[0.19796014944401197, 0.6140323903868259]"
1,1,1,[2212],[1],[0.9725774517657677],[0.1263301507117572],[0.19938262288017303],[0.09919508260842358]
2,2,2,"[2112, 111]","[1, 2]","[1.1118738559110037, 0.26367230653318857]","[-0.37268949030114623, -0.0217469271935761]","[0.12064437792810033, 0.07995593610690205]","[0.44724465792891954, -0.21080446419342747]"
3,3,2,"[2212, 211]","[2, 2]","[0.9947425097045423, 0.33418229063236526]","[0.1359989998127492, 0.10373864951607942]","[-0.23426565972115085, 0.15065100688841299]","[0.18916140027650094, 0.24236446717113797]"
4,4,1,[2212],[1],[0.9788680916071129],[-0.1595306987006702],[-0.22535854804818672],[0.039896413074052006]
5,5,3,"[2112, 111, 111]","[1, 2, 2]","[1.1786630134557343, 0.32716008581848793, 0.23...","[-0.24773881057001107, -0.16275019858550982, 0...","[0.27303580841489067, 0.1356730302253644, -0.0...","[0.6087201552303894, 0.20957155611116918, -0.0..."
6,6,1,[2212],[2],[0.9750088500743606],[-0.13935469153763638],[-0.160346926829927],[0.15860924544901556]
7,7,2,"[2212, -211]","[1, 1]","[1.6538507098452568, 0.31708058020849444]","[-0.08750094945471691, 0.22525150096807683]","[-0.015516544631886162, -0.028576637929007824]","[1.35903292570813, 0.1717713488228637]"
8,8,4,"[111, 111, 2212, 111]","[2, 1, 1, 4]","[0.37047206161945667, 1.164044835061437, 2.291...","[0.019229258834476813, -0.7193155149876883, 0....","[-0.3434288307375938, 0.12046501043563534, 0.3...","[-0.026773803758052588, 0.8971366294573256, 1...."


In [52]:
gst_ipart_df = pd.DataFrame(columns=["iev","ipart","pdg","resc","E","px","py","pz"])

In [53]:
for row in gst_df[["iev","ni","pdgi","Ei","pxi","pyi","pzi","resc"]][0:10].itertuples(index=False):
#    print(row[0],row[1],row[2])
    iev = np.full(row[1],row[0])
    ipart = np.arange(row[1])
#    print(len(iev),len(ipart))
    gst_ipart_df = pd.concat([gst_ipart_df,
                              pd.DataFrame({"iev":np.full(row[1],row[0]),
                                            "ipart":np.arange(row[1]),
                                            "pdg":row[2],
                                            "E":row[3],
                                            "px":row[4],
                                            "py":row[5],
                                            "pz":row[6],
                                            "resc":row[7]})])
print(gst_ipart_df)

  iev ipart   pdg resc         E        px        py        pz
0   0     0  2112    1  0.961755  0.045732 -0.030153  0.197960
1   0     1   111    7  0.655507 -0.144667 -0.116219  0.614032
0   1     0  2212    1  0.972577  0.126330  0.199383  0.099195
0   2     0  2112    1  1.111874 -0.372689  0.120644  0.447245
1   2     1   111    2  0.263672 -0.021747  0.079956 -0.210804
0   3     0  2212    2  0.994743  0.135999 -0.234266  0.189161
1   3     1   211    2  0.334182  0.103739  0.150651  0.242364
0   4     0  2212    1  0.978868 -0.159531 -0.225359  0.039896
0   5     0  2112    1  1.178663 -0.247739  0.273036  0.608720
1   5     1   111    2  0.327160 -0.162750  0.135673  0.209572
2   5     2   111    2  0.238910  0.158513 -0.088161 -0.077204
0   6     0  2212    2  0.975009 -0.139355 -0.160347  0.158609
0   7     0  2212    1  1.653851 -0.087501 -0.015517  1.359033
1   7     1  -211    1  0.317081  0.225252 -0.028577  0.171771
0   8     0   111    2  0.370472  0.019229 -0.343429 -0

In [58]:
gst_ipart_df = \
pd.concat([pd.DataFrame({"iev":np.full(row[1],row[0]),
                                            "ipart":np.arange(row[1]),
                                            "pdg":row[2],
                                            "E":row[3],
                                            "px":row[4],
                                            "py":row[5],
                                            "pz":row[6],
                                            "resc":row[7]}) 
           for row in gst_df[["iev","ni","pdgi","Ei","pxi","pyi","pzi","resc"]].itertuples(index=False)],
         ignore_index=True)

In [59]:
gst_ipart_df

,iev,ipart,pdg,E,px,py,pz,resc
0,0,0,2112,0.961755,0.045732,-0.030153,0.197960,1
1,0,1,111,0.655507,-0.144667,-0.116219,0.614032,7
2,1,0,2212,0.972577,0.126330,0.199383,0.099195,1
3,2,0,2112,1.111874,-0.372689,0.120644,0.447245,1
4,2,1,111,0.263672,-0.021747,0.079956,-0.210804,2
...,...,...,...,...,...,...,...,...
1740088,999995,0,2212,0.985852,-0.079529,-0.068857,0.283695,3
1740089,999996,0,2212,0.980588,0.227694,-0.093408,0.143630,2
1740090,999997,0,2212,0.991671,0.060129,-0.306679,-0.073415,2
1740091,999998,0,2212,0.996182,0.035054,0.150963,0.296659,1


In [10]:
gst_df = create_base_df("/Users/wketchum/Data/LDMX/gntp.0.gst.root",
                        max_entries=-1)

Reading -1 entries from root file /Users/wketchum/Data/LDMX/gntp.0.gst.root
Adding lepton quantities.
Adding hadron quantities.


/Library/Python/3.8/site-packages/numpy/core/_asarray.py:83: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  return array(a, dtype, copy=False, order=order)


Adding hadron acceptances.


In [11]:
knob_names = ["FrCEx_N","FrAbs_N","FrInel_N","FrPiProd_N",#"MFP_N",
              "FrCEx_pi","FrAbs_pi","FrInel_pi","FrPiProd_pi",#"MFP_pi",
              "FormZone"]

In [12]:
for kn in knob_names:
    gst_df = merge_weights_simple(filepath="/Users/wketchum/Data/LDMX",knob_name=kn,df=gst_df,qual="FSIFix")

In [13]:
gst_df

,iev,neu,fspl,tgt,Z,A,hitnuc,hitqrk,resid,sea,...,wght_FrInel_pi_p1,wght_FrInel_pi_p2,wght_FrPiProd_pi_n2,wght_FrPiProd_pi_n1,wght_FrPiProd_pi_p1,wght_FrPiProd_pi_p2,wght_FormZone_n2,wght_FormZone_n1,wght_FormZone_p1,wght_FormZone_p2
0,0,11,11,1000220480,22,48,2112,0,0,False,...,0.926723,0.853446,0.6,0.8,1.2,1.4,1.0,1.0,1.0,1.0
1,1,11,11,1000220480,22,48,2212,0,-99,False,...,1.000000,1.000000,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
2,2,11,11,1000220480,22,48,2112,0,0,False,...,0.865019,0.730039,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
3,3,11,11,1000220480,22,48,2212,0,6,False,...,0.779041,0.585628,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
4,4,11,11,1000220480,22,48,2212,0,-99,False,...,1.000000,1.000000,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
999995,999995,11,11,1000220480,22,48,2212,0,-99,False,...,1.000000,1.000000,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
999996,999996,11,11,1000220480,22,48,2212,0,-99,False,...,1.000000,1.000000,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
999997,999997,11,11,1000220480,22,48,2212,0,-99,False,...,1.000000,1.000000,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
999998,999998,11,11,1000220480,22,48,2212,0,-99,False,...,1.000000,1.000000,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0


In [14]:
import importlib
import pickle


class PickleProtocol:
    def __init__(self, level):
        self.previous = pickle.HIGHEST_PROTOCOL
        self.level = level

    def __enter__(self):
        importlib.reload(pickle)
        pickle.HIGHEST_PROTOCOL = self.level

    def __exit__(self, *exc):
        importlib.reload(pickle)
        pickle.HIGHEST_PROTOCOL = self.previous


def pickle_protocol(level):
    return PickleProtocol(level)

In [15]:
with pickle_protocol(4):
    gst_df.to_hdf("/Users/wketchum/Data/LDMX/base_generation_with_weights_9Jun_FSIFix.hdf","gst_df")

/Library/Python/3.8/site-packages/pandas/core/generic.py:2606: PerformanceWarning: 
your performance may suffer as PyTables will pickle object types that it cannot
map directly to c-types [inferred_type->mixed,key->block4_values] [items->Index(['pdgi', 'resc', 'Ei', 'pxi', 'pyi', 'pzi', 'pdgf', 'Ef', 'pxf', 'pyf',
       'pzf', 'pf', 'cthf', 'thetaxzf', 'thetayzf', 'thetazf', 'kef',
       'accept_n', 'accept_p'],
      dtype='object')]

  pytables.to_hdf(


In [62]:
gst = uproot.open("/Users/wketchum/Data/LDMX/gntp.0.gst.root:gst")

FileNotFoundError: [Errno 2] No such file or directory: '/Users/wketchum/Data/LDMX/gntp.0.gst.root:gst'

In [61]:
file.keys()

[b'gst;16', b'gst;15']